In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# https://www.statsmodels.org/stable/index.html
import statsmodels.api as sm

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from functools import partial

In [ ]:
from dotenv import load_dotenv

from pathlib import Path

env_path = Path("../../.env-live")

if env_path.exists():
    print('envs Loaded')
    load_dotenv(dotenv_path=env_path)
from jrjModelRegistry.jrjModelRegistry import registerAJrjModel

In [ ]:
sallaryMisDf = pd.read_excel("https://www.dropbox.com/scl/fi/zt2vtwhpz8ndblsxqdqx1/Salary_MIS.xlsx?rlkey=2uk6m7m9w90isv6zsynhhhpyv&st=gxumjns5&dl=1")

In [ ]:
sallaryMisDf

In [ ]:
sallaryMisDf.shape

In [ ]:
sallaryMisDf.describe()

In [ ]:
sallaryMisDf.shape

In [ ]:
sm.add_constant(sallaryMisDf[["GPA", "MIS", "Statistics"]])

In [ ]:
salaryRegModel1 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(sallaryMisDf[["GPA", "MIS", "Statistics"]])
)

In [ ]:
salaryRegModel1Fit = salaryRegModel1.fit()

In [ ]:
print(salaryRegModel1Fit.summary())

In [ ]:
sallaryMisDf

In [ ]:
salaryRegModel1SampleData = {
    "GPA": [3.53, 3.04, 3.65],
    "MIS": [1, 1, 0],
    "Statistics": [0,1, 0]
}

In [ ]:
def salaryRegModel1Transformer(dataForTransfer = None):
    import pandas as pd
    import statsmodels.api as sm
    dfTransformer = pd.DataFrame(dataForTransfer)
    dfTransformer = sm.add_constant(dfTransformer[['GPA', 'MIS', 'Statistics']])
    return dfTransformer

In [ ]:
def salaryRegModel1MainPredictor(self, transformedData):
    return self.predict(transformedData)

In [ ]:
salaryRegModel1Fit.mainPredictor = partial(salaryRegModel1MainPredictor, salaryRegModel1Fit)

In [ ]:
salaryRegModel1Fit.transformer = salaryRegModel1Transformer

In [ ]:
test1 = salaryRegModel1Fit.transformer(salaryRegModel1SampleData)
test1
# salaryRegModel1Fit.mainPredictor(test1)

In [ ]:
salaryRegModel1Fit.mainPredictor(test1)

In [ ]:
salaryRegModel1FitMetadata = {
    "modelName":f"salaryRegModel1Fit",
    "version":"1.0.1",
    "params": salaryRegModel1Fit.params.to_dict(),
    "score": float(salaryRegModel1Fit.rsquared),
    "modelLibraray": 'sm.OLS',
    "libraryMetadata": {
        "pvalues": salaryRegModel1Fit.pvalues.to_dict(),
        "r_squared": float(salaryRegModel1Fit.rsquared),
        "adj_r_squared": float(salaryRegModel1Fit.rsquared_adj)
    },

    "sampleData": {
        "dataForTransfer": salaryRegModel1SampleData
    }
}
salaryRegModel1FitMetadata

In [ ]:
registerAJrjModel(salaryRegModel1Fit, salaryRegModel1FitMetadata)

In [ ]:
sallaryMisDf

In [ ]:
sallaryMisDf["misXStatistics"] = sallaryMisDf["MIS"] * sallaryMisDf["Statistics"]
sallaryMisDf['misXStatistics1'] = sallaryMisDf.apply(lambda row: row['MIS'] * row['Statistics'], axis=1)

sallaryMisDf

In [ ]:
salaryRegModel2 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          "MIS",
          "Statistics",
          "misXStatistics1"
      ]]
  )
)

In [ ]:
salaryRegModel2Fit = salaryRegModel2.fit()

In [ ]:
print(salaryRegModel2Fit.summary())

In [ ]:
salaryRegModel2SampleData = salaryRegModel1SampleData

In [ ]:
def salaryRegModel2Transformer(dataForTransfer = None):
    import pandas as pd
    import statsmodels.api as sm
    dfTransformer = pd.DataFrame(dataForTransfer)
    dfTransformer['misXStatistics1'] = dfTransformer.apply(lambda row: row['MIS'] * row['Statistics'], axis=1)
    dfTransformer = sm.add_constant(dfTransformer[['GPA', 'MIS', 'Statistics', 'misXStatistics1']])
    return dfTransformer

In [ ]:
salaryRegModel2Fit.transformer = salaryRegModel2Transformer

In [ ]:
salaryRegModel2Fit.mainPredictor = partial(salaryRegModel1MainPredictor, salaryRegModel2Fit)

In [ ]:
test2 = salaryRegModel2Fit.transformer(salaryRegModel2SampleData)
test2

In [ ]:
salaryRegModel2Fit.mainPredictor(test2)

In [ ]:
salaryRegModel2FitMetadata = {
    "modelName":f"salaryRegModel2Fit",
    "version":"1.0.1",
    "params": salaryRegModel2Fit.params.to_dict(),
    "score": float(salaryRegModel2Fit.rsquared),
    "modelLibraray": 'sm.OLS',
    "libraryMetadata": {
        "pvalues": salaryRegModel2Fit.pvalues.to_dict(),
        "r_squared": float(salaryRegModel2Fit.rsquared),
        "adj_r_squared": float(salaryRegModel2Fit.rsquared_adj)
    },

    "sampleData": {
        "dataForTransfer": salaryRegModel2SampleData
    }
}
salaryRegModel2FitMetadata

In [ ]:
registerAJrjModel(salaryRegModel2Fit, salaryRegModel2FitMetadata)

In [ ]:
sallaryMisDf['misXGpa'] = sallaryMisDf.apply(lambda row: row['MIS'] * row['GPA'], axis=1)

sallaryMisDf


In [ ]:
salaryRegModel3 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          "MIS",
          "Statistics",
          "misXGpa"
      ]]
  )
)

In [ ]:
salaryRegModel3Fit = salaryRegModel3.fit()

In [ ]:
print(salaryRegModel3Fit.summary())

In [ ]:
salaryRegModel3Fit = salaryRegModel3.fit()

In [ ]:
print(salaryRegModel3Fit.summary())

In [ ]:
sallaryMisDf['statisticsXGpa'] = sallaryMisDf.apply(lambda row: row['Statistics'] * row['GPA'], axis=1)

sallaryMisDf

In [ ]:
salaryRegModel4 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          "MIS",
          "Statistics",
          "statisticsXGpa"
      ]]
  )
)

In [ ]:
salaryRegModel4Fit = salaryRegModel4.fit()

In [ ]:
print(salaryRegModel4Fit.summary())

In [ ]:
salaryRegModel5 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          "MIS",
          "Statistics",
          "misXStatistics1",
          "misXGpa",
          "statisticsXGpa"
      ]]
  )
)

In [ ]:
salaryRegModel5Fit = salaryRegModel5.fit()

In [ ]:
print(salaryRegModel5Fit.summary())

In [ ]:
salaryRegModel6 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          # "MIS",
          "Statistics",
          "misXStatistics1",
          # "misXGpa",
          # "statisticsXGpa"
      ]]
  )
)

In [ ]:
salaryRegModel6Fit = salaryRegModel6.fit()

In [ ]:
print(salaryRegModel6Fit.summary())

## AFTER AREFULLY LOOKUP WE RELIZED  salaryRegModel4Fit SO LETS KEEP IT AS THE BEST

In [ ]:
def salaryRegModel4Transformer(dataForTransfer = None):
    import pandas as pd
    import statsmodels.api as sm
    dfTransformer = pd.DataFrame(dataForTransfer)
    dfTransformer['statisticsXGpa'] = dfTransformer.apply(lambda row: row['Statistics'] * row['GPA'], axis=1)
    dfTransformer = sm.add_constant(dfTransformer[['GPA', 'MIS', 'Statistics', 'statisticsXGpa']])
    return dfTransformer

In [ ]:
salaryRegModel4Fit.transformer = salaryRegModel4Transformer

In [ ]:
salaryRegModel4Fit.mainPredictor = partial(salaryRegModel1MainPredictor, salaryRegModel4Fit)

In [ ]:
test4 = salaryRegModel4Fit.transformer(salaryRegModel2SampleData)
test4

In [ ]:
salaryRegModel4Fit.mainPredictor(test4)

In [ ]:
salaryRegModel4FitMetadata = {
    "modelName":f"salaryRegModel4Fit",
    "version":"1.0.1",
    "params": salaryRegModel4Fit.params.to_dict(),
    "score": float(salaryRegModel4Fit.rsquared),
    "modelLibraray": 'sm.OLS',
    "libraryMetadata": {
        "pvalues": salaryRegModel4Fit.pvalues.to_dict(),
        "r_squared": float(salaryRegModel4Fit.rsquared),
        "adj_r_squared": float(salaryRegModel4Fit.rsquared_adj)
    },

    "sampleData": {
        "dataForTransfer": salaryRegModel2SampleData
    }
}
salaryRegModel4FitMetadata

In [ ]:
registerAJrjModel(salaryRegModel4Fit, salaryRegModel4FitMetadata)

In [ ]:
sallaryMisDf
